In [1]:
import torch
import torch.optim as optim
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import time

import dataset.dataset as dataset
import datasplit.datasplit as datasplit
import model.models as models
import trainer.trainer as trainer
import utils.utils as utils

torch.cuda.device_count()
cuda0 = torch.device('cuda:0')
cuda1 = torch.device('cuda:1')
cuda2 = torch.device('cuda:2')
cuda3 = torch.device('cuda:3')
device = torch.device(cuda0 if torch.cuda.is_available() else "cpu")

# INIT

In [2]:
# transforms
transform = transforms.Compose([
        transforms.ToTensor(),
    ])

# dataset
root = '/Volumes/Macintosh HD/DATASETS/GUITAR-FX/Poly_Discrete'
excl_folders = ['TS9','MT2']
spectra_folder= 'mel_22050_1024_512'
proc_settings_csv = 'proc_settings.csv'
max_num_settings=3

dataset = dataset.FxDataset(root=root,
                            excl_folders=excl_folders, 
                            spectra_folder=spectra_folder, 
                            processed_settings_csv=proc_settings_csv,
                            max_num_settings=max_num_settings,
                            transform=transform)
dataset.init_dataset()
# dataset.generate_mel()

# split
# set test_train_split=0.0 and val_train_split=0.0 to test pre-trained model
split = datasplit.DataSplit(dataset, test_train_split=0.8, val_train_split=0.1, shuffle=True)

# loaders
train_loader, val_loader, test_loader = split.get_split(batch_size=100)

print('dataset size: ', len(dataset))
print('train set size: ', len(split.train_sampler))
print('val set size: ', len(split.val_sampler))
print('test set size: ', len(split.test_sampler))
dataset.fx_to_label

dataset size:  74760
train set size:  53827
val set size:  5981
test set size:  14952


{'808': 0,
 'BD2': 1,
 'BMF': 2,
 'DPL': 3,
 'DS1': 4,
 'FFC': 5,
 'MGS': 6,
 'OD1': 7,
 'RAT': 8,
 'RBM': 9,
 'SD1': 10,
 'VTB': 11}

# TRAIN FxNET

In [3]:
# model
fxnet = models.FxNet(n_classes=dataset.num_fx).to(device)
# optimizer
optimizer_fxnet = optim.Adam(fxnet.parameters(), lr=0.001)
# loss function
loss_func_fxnet = nn.CrossEntropyLoss()

print(fxnet)
print('Trainable Params: ', sum(p.numel() for p in fxnet.parameters() if p.requires_grad))

FxNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (batchNorm1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (batchNorm2): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=6264, out_features=120, bias=True)
  (batchNorm3): BatchNorm1d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (batchNorm4): BatchNorm1d(60, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (out): Linear(in_features=60, out_features=12, bias=True)
)
Trainable Params:  762156


In [4]:
# SAVE
models_folder = '../../models_and_results/models'
model_name = '20201211_fxnet_poly_disc_noTS9_best'
results_folder = '../../models_and_results/results'
results_subfolder = '20201211_fxnet_poly_disc_noTS9'

In [5]:
# TRAIN and TEST FxNet OVER MULTIPLE EPOCHS
train_set_size = len(split.train_sampler)
val_set_size = len(split.val_sampler)
test_set_size = len(split.test_sampler)

all_train_losses, all_val_losses, all_test_losses = [],[],[]
all_train_correct, all_val_correct, all_test_correct = [],[],[]
all_train_results, all_val_results, all_test_results = [],[],[]

best_val_correct = 0
early_stop_counter = 0

start = time.time()

for epoch in range(100):
    train_loss, train_correct, train_results = trainer.train_fx_net(
        model=fxnet,
        optimizer=optimizer_fxnet, 
        train_loader=train_loader, 
        train_sampler=split.train_sampler, 
        epoch=epoch, 
        device=device
    )
    
    val_loss, val_correct, val_results = trainer.val_fx_net(
        model=fxnet, 
        val_loader=val_loader, 
        val_sampler=split.val_sampler, 
        device=device
    )
    
    test_loss, test_correct, test_results = trainer.test_fx_net(
        model=fxnet, 
        test_loader=test_loader, 
        test_sampler=split.test_sampler, 
        device=device
    )
    # save model
    if val_correct > best_val_correct:
        best_val_correct = val_correct
        torch.save(fxnet, '%s/%s' % (models_folder, model_name))
        early_stop_counter = 0
        print('\n=== saved best model ===\n')
    else:
        early_stop_counter += 1
        
    # append results
    all_train_losses.append(train_loss)
    all_val_losses.append(val_loss)
    all_test_losses.append(test_loss)
    
    all_train_correct.append(train_correct)
    all_val_correct.append(val_correct)
    all_test_correct.append(test_correct)
    
    all_train_results.append(train_results)
    all_val_results.append(val_results)
    all_test_results.append(test_results)

    if early_stop_counter == 15:
        print('\n--- early stop ---\n')
        break

stop = time.time()
print(f"Training time: {stop - start}s")

Train Epoch: 0	[5000/53827 (9%)]	Total Loss: 85.1864	Avg Loss: 0.0170
Train Epoch: 0	[10000/53827 (19%)]	Total Loss: 133.4479	Avg Loss: 0.0133
Train Epoch: 0	[15000/53827 (28%)]	Total Loss: 165.3722	Avg Loss: 0.0110
Train Epoch: 0	[20000/53827 (37%)]	Total Loss: 188.3183	Avg Loss: 0.0094
Train Epoch: 0	[25000/53827 (46%)]	Total Loss: 206.4360	Avg Loss: 0.0083
Train Epoch: 0	[30000/53827 (56%)]	Total Loss: 223.8651	Avg Loss: 0.0075
Train Epoch: 0	[35000/53827 (65%)]	Total Loss: 238.4104	Avg Loss: 0.0068
Train Epoch: 0	[40000/53827 (74%)]	Total Loss: 251.2764	Avg Loss: 0.0063
Train Epoch: 0	[45000/53827 (83%)]	Total Loss: 262.4803	Avg Loss: 0.0058
Train Epoch: 0	[50000/53827 (93%)]	Total Loss: 271.9327	Avg Loss: 0.0054
====> Epoch: 0	Total Loss: 279.5417	 Avg Loss: 0.0052	Correct: 45756/53827	Percentage Correct: 85.01
====> Val Loss: 22.3026	 Avg Loss: 0.0037	Correct: 5164/5981	Percentage Correct: 86.34
====> Test Loss: 57.3588	 Avg Loss: 0.0038	Correct: 12805/14952	Percentage Correct: 8

In [7]:
# BEST RESULTS
print('Train Accuracy: ', 100 * max(all_train_correct) / train_set_size)
print('Epoch: ', np.argmax(all_train_correct))
print()
print('Val Accuracy: ', 100 * max(all_val_correct) / val_set_size)
print('Epoch: ', np.argmax(all_val_correct))
print()
print('Test Accuracy: ', 100 * max(all_test_correct) / test_set_size)
print('Epoch: ', np.argmax(all_test_correct))
print()

Train Accuracy:  99.59871439983651
Epoch:  53

Val Accuracy:  98.54539374686507
Epoch:  38

Test Accuracy:  98.72926698769395
Epoch:  30



In [8]:
# SAVE RESULTS - all losses, all correct, best results
all_train_losses_npy = np.array(all_train_losses)
all_train_correct_npy = np.array(all_train_correct)
best_train_results_npy = np.array(all_train_results[38])

all_val_losses_npy = np.array(all_val_losses)
all_val_correct_npy = np.array(all_val_correct)
best_val_results_npy = np.array(all_val_results[38])

all_test_losses_npy = np.array(all_test_losses)
all_test_correct_npy = np.array(all_test_correct)
best_test_results_npy = np.array(all_test_results[38])

fx_labels_npy = np.array(list(dataset.fx_to_label.keys()))

np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_train_losses')), arr=all_train_losses_npy)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_train_correct')), arr=all_train_correct_npy)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'best_train_results')), arr=best_train_results_npy)

np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_val_losses')), arr=all_val_losses_npy)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_val_correct')), arr=all_val_correct_npy)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'best_val_results')), arr=best_val_results_npy)

np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_test_losses')), arr=all_test_losses_npy)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_test_correct')), arr=all_test_correct_npy)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'best_test_results')), arr=best_test_results_npy)

np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'fx_labels')), arr=fx_labels_npy)